In [ ]:
from google.colab import files
uploaded = files.upload()


Saving spotify_dataset.csv to spotify_dataset.csv


In [ ]:
!pip install pandas torch sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
import pandas as pd
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sentence_transformers import SentenceTransformer, util

csv_file = "spotify_dataset.csv"
df = pd.read_csv(csv_file)

df = df.drop(columns=['link'], errors='ignore')
df = df.fillna("")
df['full_text'] = df['song'] + " by " + df['artist'] + " - " + df['text']

model = SentenceTransformer('all-MiniLM-L6-v2')

song_embeddings = model.encode(df['full_text'], convert_to_tensor=True)

def identify_song_pytorch(query):
    query_vector = model.encode([query], convert_to_tensor=True).to(device)

    similarities = torch.nn.functional.cosine_similarity(query_vector, song_embeddings).cpu().numpy()

    # Get the top 3 matches
    top_indices = similarities.argsort()[-3:][::-1]
    best_matches = df.iloc[top_indices]

    return best_matches[['song', 'artist']]


query_text = input("Enter a snippet of the song lyrics: ")
identified_songs = identify_song_pytorch(query_text)

print("\nIdentified Songs:")
print(identified_songs.to_string(index=False))


Enter a snippet of the song lyrics: Hello from the other side

Identified Songs:
             song      artist
Other Side Of You    Nazareth
     Hallo, Hallo Ace Of Base
Good Morning Girl     Journey
